In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import joblib


In [2]:
data = pd.read_csv('data/brazil_cities.csv', sep=';', decimal=',')

In [3]:
data.head(5)

,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,Pu_Bank,Pr_Assets,Pu_Assets,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,São Paulo,SP,1,11253503.0,11133776.0,119727.0,3576148.0,3548433.0,27715.0,10463636.0,...,8.0,1.947077e+13,2.893261e+12,5740995.0,1134570.0,3236.0,1.0,130.0,7.0,225.0
1,Osasco,SP,0,666740.0,664447.0,2293.0,202009.0,202009.0,NaN,616068.0,...,2.0,6.732330e+12,1.321699e+10,283641.0,73477.0,174.0,NaN,7.0,1.0,10.0
2,Rio De Janeiro,RJ,1,6320446.0,6264915.0,55531.0,2147235.0,2147235.0,NaN,5426838.0,...,5.0,2.283445e+12,9.738864e+11,2039930.0,363486.0,289.0,1.0,68.0,1.0,120.0
3,Brasília,DF,1,2570160.0,2564370.0,5790.0,774820.0,751558.0,23261.0,2450634.0,...,4.0,2.920933e+11,8.016164e+12,1288107.0,211392.0,594.0,1.0,28.0,1.0,60.0
4,Porto Alegre,RS,1,1409351.0,1403450.0,5901.0,508503.0,508503.0,NaN,1339712.0,...,3.0,2.709619e+11,4.026997e+11,608777.0,106829.0,1383.0,1.0,19.0,12.0,53.0


In [4]:
data.shape

(5576, 81)

In [5]:
keep = ['STATE', 'CAPITAL', 'IBGE_RES_POP', 'IDHM_Longevidade', 'IDHM_Educacao', 'GDP', 'COMP_TOT', 'CATEGORIA_TUR']

In [6]:
df = data[keep]

In [7]:
df = df.rename(columns={'STATE': 'state', 'CAPITAL': 'capital', 'IBGE_RES_POP': 'population',
                'IDHM_Longevidade': 'life_expectancy', 'IDHM_Educacao': 'education_index',
                'GDP': 'gdp', 'COMP_TOT': 'num_companies',
                'CATEGORIA_TUR': 'tourism_category',})

In [8]:
df.dtypes

state                object
capital               int64
population          float64
life_expectancy     float64
education_index     float64
gdp                 float64
num_companies       float64
tourism_category     object
dtype: object

In [9]:
df.isna().sum()

state                  0
capital                0
population             8
life_expectancy        8
education_index        8
gdp                    3
num_companies          3
tourism_category    2288
dtype: int64

In [10]:
df = df.dropna(subset=['population']) # Drop all NaN in target column

# Train Test Split

In [11]:
y = df['population']
features = [col for col in df if col != 'population']

In [12]:
X = df[features]

In [13]:
X.head()

,state,capital,life_expectancy,education_index,gdp,num_companies,tourism_category
0,SP,1,0.855,0.725,6.870359e+08,530446.0,A
1,SP,0,0.840,0.718,7.440269e+07,15315.0,B
2,RJ,1,0.845,0.719,3.294314e+08,190038.0,A
3,DF,1,0.873,0.742,2.354971e+08,86200.0,A
4,RS,1,0.857,0.702,7.342526e+04,80082.0,A


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4454, 7)
(1114, 7)
(4454,)
(1114,)


# Classify Columns into Categorical and Numerical

In [ ]:
cat_cols = ['state', 'capital', 'tourism_category']
num_cols = ['life_expectancy', 'education_index', 'gdp', 'num_companies']

# Build a Pipeline for Categorical Columns

In [ ]:
cat_cols

In [ ]:
X_train_cat = X_train[cat_cols]
X_train_cat.head(3)

In [ ]:
X_train_cat.isna().sum()

In [ ]:
si = SimpleImputer(strategy='constant', fill_value='other')

si.fit(X_train_cat)
X_train_cat_si = si.transform(X_train_cat)

In [ ]:
ohe = OneHotEncoder(sparse=False)

X_train_cat_ohe = ohe.fit_transform(X_train_cat_si)

In [ ]:
steps = [
    ('impute', si),
    ('ohe', ohe)
]

cat_pipe = Pipeline(steps=steps)
cat_pipe.fit(X_train_cat, y_train)

# Build a Pipeline for Numerical Columns

In [ ]:
num_cols

In [ ]:
X_train_num = X_train[num_cols]
X_train_num.head(3)

In [ ]:
si = SimpleImputer(strategy='mean')
ss = StandardScaler()

steps = [
    ('impute', si),
    ('ss', ss)
]

num_pipe = Pipeline(steps=steps)
num_pipe.fit(X_train_num, y_train)

# Use ColumnTransformer to Concatenate all Data Together

In [ ]:
transformers = [
    ('cat', cat_pipe, cat_cols),
    ('num', num_pipe, num_cols)
]

ct = ColumnTransformer(transformers=transformers)

In [ ]:
X_train_trans = ct.fit_transform(X_train)

In [ ]:
X_train_trans.shape

# Create a Final Pipeline with Machine Learning Model

In [ ]:
lr = LinearRegression()

final_steps = [
    ('transformer', ct),
    ('model', lr)
]

pipe = Pipeline(steps=final_steps)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)